In [ ]:
! pip install -r requirements.txt --quiet

# Semantic Kernel Sequential Orchestration

In sequential orchestration, agents are organized in a pipeline. Each agent processes the task in turn, passing its output to the next agent in the sequence. 

🔗 [Sequential Orchestration](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/agent-orchestration/sequential?pivots=programming-language-python)

In [17]:
from semantic_kernel.agents import SequentialOrchestration
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent,ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from dotenv import load_dotenv
from os import environ
import asyncio
from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin

load_dotenv(override=True)

kernel = Kernel()


In [18]:
chatCompletion = AzureChatCompletion(
    service_id="chat",
    deployment_name=environ["AZURE_OPENAI_MODEL"],
    base_url=environ["AZURE_OPENAI_ENDPOINT"],
    api_key=environ["AZURE_OPENAI_API_KEY"] )


reasoningCompletion = AzureChatCompletion(
    service_id="reasoning",
    deployment_name='o3-mini',
    base_url=environ["AZURE_OPENAI_ENDPOINT"],
    api_key=environ["AZURE_OPENAI_API_KEY"] )



In [19]:
sales_plugin = MCPStreamableHttpPlugin(
    name="sales",
    url=f"{environ['MCP_SERVER_URL']}",
)

await sales_plugin.connect()


In [20]:

# Sales Data Agent
sales_data_agent = ChatCompletionAgent(
    name="SalesDataAgent",
    instructions=(
        """You are a Sales Data Agent. Your job is to:
        - Dynamically choose and call the appropriate tool(s) to answer user questions about sales, customers, or products.
        - Always include customer names alongside IDs in any output.
        - Always display monetary amounts in $USD (e.g., $123.45).
        - Use filters and aggregations as needed to generate insights from sales orders, products, and customers.
        - Compute totals, revenue, discounts, and other metrics from nested order data.
        - Return **complete, detailed outputs** without summarization, so that all data is available for downstream analysis.
        - Preserve all intermediate results and raw values in your response.
        - Avoid hardcoding analytics; rely on the MCP tools and their parameters.
        - Clarify ambiguous queries before performing analysis.
        - Do not ask the user follow-up questions; only return the requested records.
        - Treat the tools as the source of truth; do not expose raw database internals.
        - return output in table format when possible for consumption by the next agent.
        """
    ),
    plugins=[sales_plugin],
    service=chatCompletion,
)

# Data Analysis Agent
data_analysis_agent = ChatCompletionAgent(
    name="DataAnalysisAgent",
    instructions=(
        """You are a Data Analysis Agent. Given the sales data, your job is to:
        - Analyze and interpret the sales data provided by the SalesDataAgent.
        - Identify trends, patterns, and insights from the sales data.
        - Provide actionable recommendations based on the analysis.
        - Summarize key findings in a clear and concise manner.
        - Ensure that all analyses are relevant to the user's query and objectives.
        - Provide Key Findings, Insights, Recommendations and a Summary in your final output.
        """
    ),
      service=reasoningCompletion,
)

### Optional: Observe Agent Responses

In [21]:
from semantic_kernel.contents import ChatMessageContent

def agent_response_callback(message: ChatMessageContent) -> None:
    print(f"# {message.name}\n{message.content}")

### Set Up the Sequential Orchestration

In [22]:
agents = [sales_data_agent, data_analysis_agent]
sequential_orchestration = SequentialOrchestration(
    members=agents,
    agent_response_callback=agent_response_callback,
)

### Start the Runtime

In [23]:
from semantic_kernel.agents.runtime import InProcessRuntime

runtime = InProcessRuntime()
runtime.start()

# SalesDataAgent

# SalesDataAgent

# SalesDataAgent
The customer with the highest total sales is **GM** (Customer ID: 1). Their total sales amount to $4,060.79.

### Top 3 products purchased by GM:
| **Product Name** | **Total Units Sold** | **Total Revenue ($USD)** |
|-------------------|-----------------------|--------------------------|
| Brake Pad        | 73                  | $3,213.5                |
| Oil Filter       | 42                  | $507.75                 |
| Spark Plug       | 61                  | $342.88                 |


Task was destroyed but it is pending!
task: <Task pending name='Task-45' coro=<RunContext._run() running at c:\Users\joscholt\Documents\GitHub\Azure-AI-Foundry-Databricks-Research\.venv\Lib\site-packages\semantic_kernel\agents\runtime\in_process\in_process_runtime.py:124> wait_for=<Future pending cb=[Task.task_wakeup()]>>


# DataAnalysisAgent
### Key Findings:
1. **Top Customer**: GM (Customer ID: 1) is the highest-paying customer, with total sales of $4,060.79. Their purchases dominate the sales data.
2. **Top Product by GM**: The Brake Pad is the highest contributor to revenue from GM, with 73 units sold generating $3,213.50—about 79% of their total sales.
3. **Other Significant Products**: GM also purchased Oil Filters (42 units, $507.75) and Spark Plugs (61 units, $342.88), but these account for only 12.5% and 8.5% of their total sales, respectively.

---

### Insights:
1. **Concentration of Revenue**: The Brake Pad alone contributes a significant percentage (~79%) of GM’s total sales revenue. This indicates that GM relies heavily on this product.
2. **High Interest in Automotive Parts**: GM primarily purchased automotive parts like Brake Pads, Oil Filters, and Spark Plugs, suggesting a focus on maintenance and repair items.
3. **Potential Dependency**: GM's purchasing pattern may indicate dependency

In [24]:
orchestration_result = await sequential_orchestration.invoke(
    task="Which customer has the highest total sales, and what are the top 3 products they purchased?",
    runtime=runtime,
)

In [ ]:
value = await orchestration_result.get(timeout=30)
print(f"***** Final Result *****\n{value}")

***** Final Result *****
### Key Findings:
1. **Top Customer**: GM (Customer ID: 1) is the highest-paying customer, with total sales of $4,060.79. Their purchases dominate the sales data.
2. **Top Product by GM**: The Brake Pad is the highest contributor to revenue from GM, with 73 units sold generating $3,213.50—about 79% of their total sales.
3. **Other Significant Products**: GM also purchased Oil Filters (42 units, $507.75) and Spark Plugs (61 units, $342.88), but these account for only 12.5% and 8.5% of their total sales, respectively.

---

### Insights:
1. **Concentration of Revenue**: The Brake Pad alone contributes a significant percentage (~79%) of GM’s total sales revenue. This indicates that GM relies heavily on this product.
2. **High Interest in Automotive Parts**: GM primarily purchased automotive parts like Brake Pads, Oil Filters, and Spark Plugs, suggesting a focus on maintenance and repair items.
3. **Potential Dependency**: GM's purchasing pattern may indicate depen

In [26]:
await runtime.stop_when_idle()